In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
#allows to run tensorflow code without creating graph
tf.enable_eager_execution()
################################ STEP 1: LOADING DATASETS ###################################################
# Load Dataset
boston = load_boston()
# Seperate Data into Features and Labels
# Features
features_df = pd.DataFrame(np.array(boston.data), columns=[boston.feature_names])
# Labels
labels_df = pd.DataFrame(np.array(boston.target), columns=['labels'])
#splitting data into training, testing and validation datasets
X_train = features_df[:300]
X_val = features_df[301:401]
X_test=features_df[402:]

y_train = labels_df[:300]
y_val = labels_df[301:401]
y_test=labels_df[402:]

m, n = X_train.shape
m1, n1 = X_test.shape
m2, n2 = X_val.shape
######################################## STEP 2: PREPROCESSING DATA #########################################################
# scaling and normalizing data values
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(X_train)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

scaled_housing_data1 = scaler.fit_transform(X_test)
scaled_housing_data_plus_bias1 = np.c_[np.ones((m1, 1)), scaled_housing_data1]

scaled_housing_data2 = scaler.fit_transform(X_val)
scaled_housing_data_plus_bias2 = np.c_[np.ones((m2, 1)), scaled_housing_data2]

#converting to numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

X_val = np.array(X_val)
y_val = np.array(y_val)

print('Training:', X_train.shape)
print('Validation:',X_val.shape)
print('Testing:',X_test.shape)

Training: (300, 13)
Validation: (100, 13)
Testing: (104, 13)


In [3]:
################################ STEP 3: INITIALIZE VARIABLES AND CONSTANT ###################################################

#decalaring number of epochs and learning rate
n_epochs = 1000
learning_rate = 0.1
#declaring constants for training, testing and validation dataset into tensors
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
XTEST = tf.constant(scaled_housing_data_plus_bias1, dtype=tf.float32, name="XTEST")
XVAL = tf.constant(scaled_housing_data_plus_bias2, dtype=tf.float32, name="XVAL")
y = tf.constant(y_train.reshape(-1, 1), dtype=tf.float32, name="y")
#declaring variables for weights
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
#finding the loss
error = y_pred - y
#computing mean square error ℓ(𝑤) =1/2𝑁 (Summation of [𝑡(i) − 𝑦(𝑥(i))]^2)
mse = (tf.reduce_mean(tf.square(error), name="mse"))/2
#to compute gradients
variables = [theta]
#initialize optimizer(used:AdamOptimizer)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
##################################### STEP 4: USE tf.GradientTape() ######################################################
for e in range(n_epochs):
    # Use tf.GradientTape() to record the gradients of the loss function
    with tf.GradientTape() as tape:
        y_pred =tf.matmul(X, theta, name="predictions")
        error = y_pred - y
        mse = (tf.reduce_mean(tf.square(error), name="mse"))/2
    # calculates the gradients of the loss function with respect to weights stored in variables.
    grads = tape.gradient(mse, variables)
    # updates weights based on gradients.
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))
   
    if e % 100 == 0:
         print('Epoch: {0}, Loss: {1}'.format(e, mse))

##################################### STEP 5: PRINT LOSS AND GRADIENTS ######################################################
y_predT =tf.matmul(X, theta, name="predictions1")
errorT = y_predT - y_train
# Mean Squared Error for Training 
mse = (tf.reduce_mean(tf.square(errorT), name="mse"))/2
print('Cost for Training Dataset: ',mse)

y_pred2 =tf.matmul(XVAL, theta, name="predictions1")
error2 = y_pred2 - y_val
# Mean Squared Error for Validation 
mse2 = (tf.reduce_mean(tf.square(error2), name="mse2"))/2
print('Cost for Validation Dataset: ',mse2)

y_pred1 =tf.matmul(XTEST, theta, name="predictions1")
error1 = y_pred1 - y_test
# Mean Squared Error for Testing 
mse1 = (tf.reduce_mean(tf.square(error1), name="mse1"))/2
print('Cost for Testing Dataset: ',mse1)

Epoch: 0, Loss: 358.7508239746094
Epoch: 100, Loss: 129.6981201171875
Epoch: 200, Loss: 44.12686538696289
Epoch: 300, Loss: 14.303263664245605
Epoch: 400, Loss: 6.524878025054932
Epoch: 500, Loss: 5.043664932250977
Epoch: 600, Loss: 4.839001178741455
Epoch: 700, Loss: 4.8184990882873535
Epoch: 800, Loss: 4.817017555236816
Epoch: 900, Loss: 4.8169403076171875
Cost for Training Dataset:  tf.Tensor(4.8169374, shape=(), dtype=float32)
Cost for Validation Dataset:  tf.Tensor(60.73302, shape=(), dtype=float32)
Cost for Testing Dataset:  tf.Tensor(64.86148, shape=(), dtype=float32)
